<a href="https://colab.research.google.com/github/rebecabarroso/paineis_estaticos_dinamicos_maesfam/blob/main/DID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando pacotes


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [ ]:
# Gerando dados de exemplo
np.random.seed(0)

Variáveis de controle: tamanho_empresa, setor, lucratividade e idade_media_funcionarios

In [ ]:
# Variáveis de controle
n_empresas = 100
tamanho_empresa = np.random.randint(50, 500, size=n_empresas)
setor = np.random.choice(['manufatura', 'tecnologia', 'serviços'], size=n_empresas)
lucratividade = np.random.uniform(0.05, 0.20, size=n_empresas)
idade_media_funcionarios = np.random.randint(25, 55, size=n_empresas)

Efeito do programa de treinamento com a variável efeito_programa (aumenta a produtividade em 5 unidades para as empresas no grupo de tratamento)

In [ ]:
# Grupo de tratamento (empresas que participaram do programa de treinamento)
tratamento = np.random.choice([0, 1], size=n_empresas, p=[0.5, 0.5])

In [ ]:
# Efeito do programa de treinamento
efeito_programa = 5 * tratamento

Variáveis dependentes: produtividade_antes e produtividade_depois para representar a produtividade antes e depois do programa de treinamento

In [ ]:
# Variáveis dependentes (produtividade)
produtividade_antes = np.random.normal(loc=30, scale=5, size=n_empresas)
produtividade_depois = produtividade_antes + efeito_programa + np.random.normal(loc=0, scale=2, size=n_empresas)

In [ ]:
# Criando DataFrame com os dados
data = pd.DataFrame({
    'empresa_id': range(n_empresas),
    'tratamento': tratamento,
    'tamanho_empresa': tamanho_empresa,
    'setor': setor,
    'lucratividade': lucratividade,
    'idade_media_funcionarios': idade_media_funcionarios,
    'produtividade_antes': produtividade_antes,
    'produtividade_depois': produtividade_depois
})

In [ ]:
# Aplicando dummy para setor, manufatura = 1, serviços = 2 e tecnologia = 3
data['setor'] = data['setor'].replace({'manufatura': 1, 'serviços': 2, 'tecnologia': 3})
data

,empresa_id,tratamento,tamanho_empresa,setor,lucratividade,idade_media_funcionarios,produtividade_antes,produtividade_depois,periodo,interacao_tratamento_periodo
0,0,0,222,1,0.125199,44,33.449091,36.745361,antes,0
1,1,1,97,1,0.193413,33,36.509231,41.837687,depois,1
2,2,0,167,1,0.146599,32,26.859562,27.994143,antes,0
3,3,1,242,1,0.113578,48,27.594864,32.149514,depois,1
4,4,1,373,2,0.140959,38,41.519583,45.812720,depois,1
...,...,...,...,...,...,...,...,...,...,...
95,95,0,374,2,0.098003,28,24.887466,24.038831,antes,0
96,96,1,312,3,0.107520,31,30.193153,36.918345,depois,1
97,97,1,97,1,0.138248,42,21.716424,21.405186,depois,1
98,98,1,177,1,0.174657,32,25.072446,33.099102,depois,1


In [ ]:
# Modelando o efeito causal usando OLS
data['interacao_tratamento_periodo'] = data['tratamento'] * (data['periodo'] == 'depois')

x = data[['setor','tratamento', 'interacao_tratamento_periodo', 'tamanho_empresa', 'lucratividade', 'idade_media_funcionarios']]
y = data['produtividade_depois']

In [ ]:
# Adicionar intercepto ao modelo

In [ ]:
x = sm.add_constant(x)

In [ ]:
model = sm.OLS(y, x)
results = model.fit()

print(results.summary())

                             OLS Regression Results                             
Dep. Variable:     produtividade_depois   R-squared:                       0.131
Model:                              OLS   Adj. R-squared:                  0.084
Method:                   Least Squares   F-statistic:                     2.826
Date:                  Fri, 28 Jun 2024   Prob (F-statistic):             0.0202
Time:                          17:28:49   Log-Likelihood:                -303.55
No. Observations:                   100   AIC:                             619.1
Df Residuals:                        94   BIC:                             634.7
Df Model:                             5                                         
Covariance Type:              nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------